# Ontology Search

This program will load and explore the Cell Ontology from https://www.ebi.ac.uk/ols/ontologies/cl

From the above site, you can download the ontology file as *cl.owl*.

The leafs in the ontology file contain information from the following:

- BFO: Basic Formal Ontology 
- CARO: Common Anatomy Reference Ontology
- CHEBI: Chemical Entities of Biological Interest
- CL: Cell Ontology
- CP: unclear meaning
- http: start of website, inclusion is unclear
- GO: Gene Ontology
- NCBITaxon: ncbitaxon
- PATO: Phenotype and Trait Ontology
- PR: Protein Ontology
- RO: Relations Ontology
- UBERON: Uber-anatomy ontology



To acess the ontology, we will use *pronto*, which can be installed with *pip*. Some resources:

- From https://github.com/althonos/pronto
- DOI: https://doi.org/10.5281/zenodo.595572
- PDF of release notes: https://pronto.readthedocs.io/_/downloads/en/latest/pdf/

In [1]:
from pronto import Ontology

import json
import urllib.request

import pandas as pd

In [2]:
cl = Ontology.from_obo_library('cl.owl')

/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:289: SyntaxWarning: <Element '{http://purl.org/dc/elements/1.1/}description' at 0x7f9eaa58b180> contains text but no `xsd:datatype`
  meta.annotations.add(self._extract_literal_pv(child))
/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:289: SyntaxWarning: <Element '{http://purl.org/dc/elements/1.1/}title' at 0x7f9eaa58b3b0> contains text but no `xsd:datatype`
  meta.annotations.add(self._extract_literal_pv(child))
/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:289: SyntaxWarning: <Element '{http://www.w3.org/2002/07/owl#}versionInfo' at 0x7f9eaa58ba40> contains text but no `xsd:datatype`
  meta.annotations.add(self._extract_literal_pv(child))
/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:113: SyntaxWarning: unknown elem

/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:117: SyntaxWarning: unknown element in `owl:Class`: {http://purl.obolibrary.org/obo/}RO_0004050
  self._extract_term(class_, curies)
/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:117: SyntaxWarning: unknown element in `owl:Class`: {http://purl.org/dc/terms/}contributor
  self._extract_term(class_, curies)
/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:117: SyntaxWarning: unknown element in `owl:Class`: {http://xmlns.com/foaf/0.1/}depicted_by
  self._extract_term(class_, curies)
/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/parsers/rdfxml.py:117: SyntaxWarning: unknown element in `owl:Class`: {http://purl.obolibrary.org/obo/}IAO_0000119
  self._extract_term(class_, curies)
/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/

/Users/josh.fuchs/My Drive/Personal/michigan/venv/lib/python3.8/site-packages/pronto/ontology.py:283: SyntaxWarning: unknown axiom property: 'http://purl.obolibrary.org/obo/UBPROP_0000004'
  cls(self).parse_from(_handle)  # type: ignore


In [3]:
cl

Ontology('http://purl.obolibrary.org/obo/cl.owl')

In [4]:
cl['CL:0002116']

Term('CL:0002116', name='B220-low CD38-positive unswitched memory B cell')

### Identify all Leaf Nodes

In [5]:
leaf_list = []

for term in cl.terms():
    if term.is_leaf():
        leaf_list.append(term.id)

In [6]:
leaf_list

['BFO:0000006',
 'BFO:0000019',
 'BFO:0000031',
 'BFO:0000034',
 'CARO:0000004',
 'CARO:0000010',
 'CARO:0000012',
 'CARO:0000013',
 'CARO:0000014',
 'CARO:0000032',
 'CARO:0000040',
 'CARO:0000042',
 'CARO:0000054',
 'CHEBI:14321',
 'CHEBI:15355',
 'CHEBI:15377',
 'CHEBI:15379',
 'CHEBI:15440',
 'CHEBI:16336',
 'CHEBI:16480',
 'CHEBI:16541',
 'CHEBI:16646',
 'CHEBI:16670',
 'CHEBI:16796',
 'CHEBI:16990',
 'CHEBI:16991',
 'CHEBI:17026',
 'CHEBI:17087',
 'CHEBI:17089',
 'CHEBI:17234',
 'CHEBI:17347',
 'CHEBI:17478',
 'CHEBI:17544',
 'CHEBI:18059',
 'CHEBI:18085',
 'CHEBI:18133',
 'CHEBI:18154',
 'CHEBI:18421',
 'CHEBI:22868',
 'CHEBI:23897',
 'CHEBI:23906',
 'CHEBI:23965',
 'CHEBI:24261',
 'CHEBI:24318',
 'CHEBI:24402',
 'CHEBI:24431',
 'CHEBI:24505',
 'CHEBI:24780',
 'CHEBI:24828',
 'CHEBI:24835',
 'CHEBI:24867',
 'CHEBI:24875',
 'CHEBI:24943',
 'CHEBI:25354',
 'CHEBI:25805',
 'CHEBI:25806',
 'CHEBI:25905',
 'CHEBI:26020',
 'CHEBI:26650',
 'CHEBI:26764',
 'CHEBI:27584',
 'CHEBI:28087',

In [7]:
# identify starting identifies of all leafs

unique_identifiers = []
             
for x in leaf_list:
    split_leaf = x.split(':')
    if split_leaf[0] not in unique_identifiers:
        unique_identifiers.append(split_leaf[0])

print(unique_identifiers)    

['BFO', 'CARO', 'CHEBI', 'CL', 'CP', 'http', 'GO', 'NCBITaxon', 'PATO', 'PR', 'RO', 'UBERON']


In [22]:
#cl.get_relationship("CL:0002116")
#cl["CL:0002116"].get_relationship()

## Identify subclasses of identifier

In [20]:
# print child terms for Leukocytes, within a distance of 2

root_node = cl['CL:0000202']

for term in root_node.subclasses(distance=1,with_self=False).to_set():
    print(term)

Term('CL:4023120', name='cochlea auditory hair cell')
Term('CL:0005008', name='macular hair cell')


## Identify superclasses of identifier



In [10]:
root_node = cl['CL:0000202']

for term in root_node.superclasses(distance=3,with_self=False).to_set():
    print(term)

Term('CL:0000199', name='mechanoreceptor cell')
Term('CL:0000855', name='sensory hair cell')
Term('CL:0000101', name='sensory neuron')


In [20]:
root_node = cl['CL:0000202']

for term in root_node.superclasses(distance=1,with_self=False).to_set():
    print(term.id)

CL:0000855


In [7]:
def find_parent(subclass):
    root_node = cl[subclass]
    parents = []
    for term in root_node.superclasses(distance=1,with_self=False).to_set():
        parents.append(term.id)
    return parents

In [8]:
find_parent('CL:0000202')

['CL:0000855']

In [9]:
find_parent('CL:0000855')

['CL:0000199']

In [10]:
find_parent('CL:0000199')

['CL:0000101']

## Get tree structure for particular cells

In [53]:
def determine_tree(start_cell,distance):
    root_node = start_cell
    
    root_iter = root_node
    calculated_distance = 0
    while calculated_distance < distance:
        # print one level down
        print('Subclasses of ', root_iter.id, 'are:')
        for term in root_iter.subclasses(distance=1,with_self=False).to_set():
            print(term.id)
            #root_iter = cl[term.id]
        calculated_distance += 1
            
        # determine number of subclasses for this id
        subclass_len_by_term = len(root_iter.subclasses(distance=1,with_self=False).to_set())
        subclasses_by_term = list(root_node.subclasses(distance=1,with_self=False).to_set().ids)
        for sub_dis in subclasses_by_term:
            print('Subclasses of ', sub_dis, 'are:')
            new_root = cl[sub_dis]
            for term in new_root.subclasses(distance=1,with_self=False).to_set():
                print(term.id) 
                
        calculated_distance += 1
        
#     print('Superclasses are:')
#     for term in root_node.superclasses(distance=distance,with_self=False).to_set():
#         print(term) 
    

In [54]:
determine_tree(cl['CL:0000202'],1)

Subclasses of  CL:0000202 are:
CL:4023120
CL:0005008
Subclasses of  CL:4023120 are:
CL:0000589
CL:0000601
Subclasses of  CL:0005008 are:


In [51]:
root_node = cl['CL:0000202']

for term in root_node.subclasses(distance=None,with_self=False).to_set():
    print(term)

Term('CL:4023120', name='cochlea auditory hair cell')
Term('CL:0000589', name='cochlear inner hair cell')
Term('CL:0000601', name='cochlear outer hair cell')
Term('CL:0005008', name='macular hair cell')


In [36]:
len(root_node.subclasses(distance=1,with_self=False).to_set())

2

In [39]:
list(root_node.subclasses(distance=1,with_self=False).to_set().ids)

['CL:0000589', 'CL:0000601']

In [62]:
root_node = cl['CL:0000542'].subclasses().to_set()
data = []

for term in root_node:
    value = {'id': int(term.id[3:]), 'name': term.id, 'desc': term.name}
    parents = term.superclasses(with_self=False, distance=1).to_set() & root_node
    if parents:
        value['parent'] = int(parents.pop().id[3:])
    data.append(value)

In [63]:
data

[{'id': 2115,
  'name': 'CL:0002115',
  'desc': 'B220-positive CD38-positive unswitched memory B cell',
  'parent': 2114},
 {'id': 2122,
  'name': 'CL:0002122',
  'desc': 'B220-positive CD38-positive IgG-negative class switched memory B cell',
  'parent': 2119},
 {'id': 922, 'name': 'CL:0000922', 'desc': 'type II NK T cell', 'parent': 814},
 {'id': 1048,
  'name': 'CL:0001048',
  'desc': 'activated CD4-positive, CD25-positive, CCR4-positive, alpha-beta regulatory T cell, human',
  'parent': 1047},
 {'id': 919,
  'name': 'CL:0000919',
  'desc': 'CD8-positive, CD25-positive, alpha-beta regulatory T cell',
  'parent': 795},
 {'id': 9045,
  'name': 'CL:0009045',
  'desc': 'B cell of medullary sinus of lymph node',
  'parent': 236},
 {'id': 2444,
  'name': 'CL:0002444',
  'desc': 'Ly49H-positive natural killer cell, mouse',
  'parent': 2438},
 {'id': 2432,
  'name': 'CL:0002432',
  'desc': 'CD24-positive, CD4 single-positive thymocyte',
  'parent': 810},
 {'id': 2407,
  'name': 'CL:0002407'

In [61]:
# possible way to view, from page 10-11 of https://pronto.readthedocs.io/_/downloads/en/latest/pdf/

# Let's use the Vega radial tree example as a basis of the visualization
view = json.load(urllib.request.urlopen("https://vega.github.io/vega/examples/radial-tree-layout.vg.json"))

# First replace the default data with our own
view['data'][0].pop('url')
view['data'][0]['values'] = data

view['marks'][1]['encode']['enter']['tooltip'] = {"signal": "datum.desc"}
view['signals'][4]['value'] = 'cluster'
# Render the clustered tree
display({"application/vnd.vega.v5+json": view}, raw=True)

## Identify leafs under a particular cell ID

Idenfity the leafs under a particular cell ID. This will return a list of leaf cell ontologies.

In [5]:
root_node = cl['CL:0000542'] # lymphocyte

leaf_list = []

for term in root_node.subclasses(distance=None,with_self=False).to_set():
    if term.is_leaf():
        leaf_list.append(term.id)

In [6]:
leaf_list

['CL:0009018',
 'CL:0009113',
 'CL:0002106',
 'CL:0000935',
 'CL:0002048',
 'CL:0000949',
 'CL:0002126',
 'CL:0000805',
 'CL:0009060',
 'CL:0000951',
 'CL:0000986',
 'CL:0002039',
 'CL:0000969',
 'CL:0000902',
 'CL:0002514',
 'CL:0001048',
 'CL:0002051',
 'CL:0000794',
 'CL:0000958',
 'CL:0000963',
 'CL:0000901',
 'CL:0002409',
 'CL:0002108',
 'CL:0002345',
 'CL:0002050',
 'CL:0002432',
 'CL:0000917',
 'CL:0009082',
 'CL:0002343',
 'CL:0009050',
 'CL:0002415',
 'CL:0002348',
 'CL:0002040',
 'CL:0001045',
 'CL:0002446',
 'CL:0002124',
 'CL:0002444',
 'CL:0000916',
 'CL:0002440',
 'CL:0001041',
 'CL:0002218',
 'CL:0002125',
 'CL:0009037',
 'CL:0000937',
 'CL:0000973',
 'CL:0001075',
 'CL:0002127',
 'CL:0002042',
 'CL:4030030',
 'CL:0000900',
 'CL:0009032',
 'CL:0000907',
 'CL:0009083',
 'CL:0000964',
 'CL:0009069',
 'CL:0000913',
 'CL:0000976',
 'CL:2000006',
 'CL:0009111',
 'CL:0002403',
 'CL:0002423',
 'CL:0000821',
 'CL:0001202',
 'CL:0001046',
 'CL:0002448',
 'CL:0000918',
 'CL:00021